In [6]:
%matplotlib inline
import importlib
from pytorch_utils_oh_2 import *

In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Data loading

In [8]:
all_data_org = pd.read_csv('data/en_train_org.csv', keep_default_na=False)

In [9]:
all_data = all_data_org.copy()
# all_data.is_copy
# all_data_org[pd.isnull(all_data_org['before'])][:3]
all_data = all_data.dropna()
print("Data rows: {},  (dropped none rows: {})".format(len(all_data), len(all_data_org)-len(all_data)))

Data rows: 9918441,  (dropped none rows: 0)


In [10]:
all_data_changed = all_data[all_data['before']!=all_data['after']]
len(all_data_changed)

659793

# Character handling

### Chars that are related to changes

In [11]:
tmp = sorted(list(set(all_data['before'] + all_data['after'])))
chars_all_weird = sorted(list(set(''.join(tmp))))
len(chars_all_weird)

3081

In [12]:
''.join(chars_all_weird[1000:1010])

'刃分刑刘刚创初別利制'

In [13]:
tmp = sorted(list(set(all_data_changed['before'] + all_data_changed['after'])))
chars_with_changes = sorted(list(set(''.join(tmp))))
len(chars_with_changes)
''.join(chars_with_changes)

154

' !"#$%&\'()+,-./0123456789:;=>ABCDEFGHIJKLMNOPQRSTUVWXYZ_abcdefghijklmnopqrstuvwxyz~£¥ª²³µº¼½¾×éɒʻˈΑΒΓΔΕΖΗΘΙΚΛΜΝΞΟΠΡΣΤΥΦΧΨΩαβγδεζηθικλμνξοπρςστυφχψω—€⅓⅔⅛⅝⅞'

In [ ]:
pickle.dump(chars_with_changes, open('data/en_features/chars_with_changes.pkl', "wb"))

In [11]:
chars_with_changes == pickle.load(open('data/en_features/chars_with_changes.pkl', "rb"))

True

### Get chars that change allways to same thing

In [ ]:
cwc_unique = {} # chars with changes unique afters
for c in chars_with_changes:
    cwc_unique[c] = len(all_data[all_data['before'].str.contains(c, regex=False)]['after'].unique())

In [ ]:
chars_with_single_output = [k for k, v in cwc_unique.items() if v <= 1]
''.join(chars_with_single_output)

In [ ]:
chars_with_single_output_dict = {}
for c in chars_with_single_output:
    tmp = all_data_changed[all_data_changed['before'].str.contains(c, regex=False)]['after'].values[0]
    chars_with_single_output_dict[c] = tmp

In [ ]:
'  ||  '.join([str(k) + " -> " + str(v) for k, v in chars_with_single_output_dict.items()])

In [ ]:
all_data[all_data['before'].str.contains('+', regex=False)]

In [ ]:
pickle.dump(chars_with_single_output_dict, open('data/en_features/chars_with_single_output_dict.pkl', "wb"))

In [ ]:
chars_with_single_output_dict == pickle.load(open('data/en_features/chars_with_single_output_dict.pkl', "rb"));

In [13]:
chars_with_single_output_dict = pickle.load(open('data/en_features/chars_with_single_output_dict.pkl', "rb"));

### Chars that have multiple usage

In [ ]:
chars_normal = set(chars_with_changes) - set(chars_with_single_output)
chars_normal = [SOS_TOKEN, EOS_TOKEN, VERBATIM_CHAR] + sorted(list(chars_normal))
''.join(chars_normal)
len(chars_normal)

In [ ]:
pickle.dump(chars_normal, open('data/en_features/chars_normal.pkl', "wb"))

In [14]:
chars_normal = pickle.load(open('data/en_features/chars_normal.pkl', "rb"));

# Words

In [133]:
words_before_pd = list(all_data['before'])

In [134]:
len(words_before_pd)

9918441

In [135]:
%%time
words_before = np.concatenate([simple_tokeniser(sent) for sent in words_before_pd])

CPU times: user 1min 44s, sys: 15.9 s, total: 2min
Wall time: 2min 58s


In [136]:
words_before[0:10]

array(['brillantaisia', 'is', 'a', 'genus', 'of', 'plant', 'in', 'family',
       'acanthaceae', '.'], 
      dtype='<U323')

In [137]:
len(words_before)
len(set(words_before))

10437549

353961

In [138]:
words_before_all_index = {}
for w in words_before:
    words_before_all_index[w] = words_before_index.get(w, 0) + 1

In [153]:
sum(words_before_all_index.values())
len(words_before_all_index)

10437549

353961

In [140]:
del(words_before_pd)
del(words_before)

In [148]:
pickle.dump(words_before_all_index, open("data/en_features/words_before_all_index.pkl", "wb"))

In [191]:
for i in [1, 5, 10, 50, 100, 250, 500, 1000, 10000]:
    tmp = [v for k, v in words_before_all_index.items() if v > i]
    print("{:>5d}: {:2.2%} (words: {})".format(i,  sum(tmp)/sum(words_before_all_index.values()), len(tmp)))

    1: 97.94% (words: 139150)
    5: 95.66% (words: 53627)
   10: 94.32% (words: 35103)
   50: 89.40% (words: 12210)
  100: 86.15% (words: 7433)
  250: 80.47% (words: 3680)
  500: 74.71% (words: 1972)
 1000: 68.12% (words: 979)
10000: 45.61% (words: 68)


In [169]:
tmp = [re.search('[0-9]', w) for w in list(words_before_index.keys())]
tmp = len(list(filter(None, tmp)))
tmp
tmp / len(words_before_index) # how many percent of individual words are number related

11991

0.03387661352521888

In [192]:
words_before_common = sorted(list(words_before_index.items()), key=lambda a: -a[1])
words_before_common = [k for k, v in words_before_common]
len(words_before_common)
words_before_common[0:10]

353961

['.', ',', 'the', '"', 'of', 'in', 'and', 'a', '(', ')']

In [193]:
pickle.dump(words_before_common, open("data/en_features/words_before_all.pkl", "wb"))

In [194]:
pickle.dump(words_before_common[0:10000], open("data/en_features/words_before_10k.pkl", "wb"))

In [196]:
tmp = [re.search('[0-9]', w) for w in words_before_common[0:10000]]
tmp = len(list(filter(None, tmp)))
tmp
tmp / len(words_before_index) # how many percent of individual words are number related in 10k most common

594

0.0016781509827353861

In [204]:
common_words_dict = dict((c, i) for i, c in enumerate(words_before_common[0:8192]))

In [209]:
common_words_dict['the']

2

# Replace verbatim infos

In [5]:
VERBATIM_CHAR

'☒'

In [220]:
len(all_data[(all_data['class'] == 'VERBATIM') & (all_data['before'].str.len() > 1)])
len(all_data[(all_data['class'] == 'VERBATIM') & (all_data['before'].str.len() <= 1)])

9002

69106

### Change only number verbatim to numbers

In [222]:
verb_numbers = (all_data['class'] == 'VERBATIM') & all_data['before'].str.contains('^[0-9]')
len(all_data.loc[verb_numbers])
all_data.loc[verb_numbers].sample(2)

21

,sentence_id,token_id,class,before,after
909252,72266,5,VERBATIM,28640,two eight six four o
4599546,352345,9,VERBATIM,10017,one o o one seven


In [224]:
all_data.loc[verb_numbers, 'class'] = 'NUMBERS'

In [226]:
len(all_data[(all_data['class'] == 'VERBATIM') & all_data['before'].str.contains('^[0-9]')])

0

### Remove verbatin data starting with '.', more errors than success

In [228]:
verb_data = all_data[all_data['class'] == 'VERBATIM']

In [237]:
verb_data[verb_data['before'].str.contains('^\.')].sample(10)
len(verb_data[verb_data['before'].str.contains('^\.')])

,sentence_id,token_id,class,before,after
4021585,308922,6,VERBATIM,.7.2,dot s e v e n dot t w o
1673641,130959,11,VERBATIM,.3.2,dot t h r e e dot t w o
209388,16889,19,VERBATIM,.2.100,dot t w o dot o n e o o
71951,5790,8,VERBATIM,.5.11,dot f i v e dot o n e o n e
9130697,689612,12,VERBATIM,.5-inch,dot f i v e d a s h i n c h
2478798,192469,5,VERBATIM,.6-mile,dot s i x d a s h m i l e
1169870,92344,2,VERBATIM,.0.0,dot o dot o
8924801,674392,17,VERBATIM,.4.3,dot f o u r dot t h r e e
7696417,583436,4,VERBATIM,.5-Inch,dot f i v e d a s h i n c h
4935763,377489,3,VERBATIM,.1-Mb,dot o n e d a s h m b


249

In [238]:
all_data[(all_data['class']=='VERBATIM') & all_data['before'].str.contains('^\.')].sample(5)
len(all_data[(all_data['class']=='VERBATIM') & all_data['before'].str.contains('^\.')])

,sentence_id,token_id,class,before,after
1274058,100316,11,VERBATIM,.0.3,dot o dot t h r e e
4127746,316925,5,VERBATIM,.0-mile,dot o d a s h m i l e
776061,61820,18,VERBATIM,.6.2p,dot s i x dot t w o p
7537703,571528,6,VERBATIM,.9.88,dot n i n e dot e i g h t e i g h t
7296864,553525,16,VERBATIM,.1.5,dot o n e dot f i v e


249

In [239]:
all_data = all_data[~((all_data['class']=='VERBATIM') & all_data['before'].str.contains('^\.'))]

In [240]:
len(all_data[(all_data['class']=='VERBATIM') & all_data['before'].str.contains('^\.')])

0

In [241]:
print("Dropped stupid verbatim infos:", len(all_data_org)-len(all_data))

Dropped stupid verbatim infos: 249


In [242]:
all_data = all_data.copy()

### Change long verbatim to letters

In [243]:
verb_letters = (all_data['class'] == 'VERBATIM') & (all_data['before'].str.len() > 1)
len(all_data.loc[verb_letters])
all_data.loc[verb_letters].sample(5)

8732

,sentence_id,token_id,class,before,after
1659990,129916,5,VERBATIM,pp,p p
2898210,224182,18,VERBATIM,cf,c f
5723150,436510,7,VERBATIM,pp,p p
6702311,509276,14,VERBATIM,NCp,n c p
3414274,263194,7,VERBATIM,pp,p p


In [244]:
chars_with_no_changes_re = re.compile("[^{}]".format(''.join(chars_with_changes)))

In [246]:
chars_with_no_changes_re.sub(VERBATIM_CHAR, "ASDFFSDA\_/변별병보복")

'ASDFFSDA☒_/☒☒☒☒☒'

In [250]:
tmp = len(all_data[(all_data['class'] == 'VERBATIM') & (all_data['before'].str.len() > 1) & (all_data['before'].str.contains(chars_with_no_changes_re))])
# 0 if all longer are really letters
tmp
tmp == 0

0

True

In [251]:
all_data.loc[verb_letters, 'class'] = 'LETTERS'

In [252]:
len(all_data[(all_data['class'] == 'VERBATIM') & (all_data['before'].str.len() > 1)])

0

### Check one letter verbatims, are they always the same

In [17]:
all_data[(all_data['class'] == 'VERBATIM')].sample(2)

,sentence_id,token_id,class,before,after
7677045,581981,73,VERBATIM,о,о
6767106,514175,2,VERBATIM,&,and


In [23]:
tmp = all_data[(all_data['class'] == 'VERBATIM')].groupby(['before', 'after']).count()

In [50]:
tmp

sentence_id  token_id  class
before after                                     
#      hash                   74        74     74
       number               1402      1402   1402
$      dollar                 44        44     44
%      percent                24        24     24
&      and                  9684      9684   9684
-      -                   11889     11889  11889
=      equals                  1         1      1
>      greater than            3         3      3
_      underscore            294       294    294
~      tilde                  63        63     63
£      pound                   1         1      1
ª      ª                       1         1      1
²      squared                16        16     16
³      cubed                   6         6      6
µ      micro                   4         4      4
¹      ¹                       2         2      2
º      º                      28        28     28
Ð      Ð                       4         4      4
×      times                   2         2      2
Ą      Ą                       1         1      1
Ď      Ď                       3         3      3
Ē      Ē                       2         2      2
Ę      Ę                       1         1      1
Ġ      Ġ                       2         2      2
Ī      Ī                       5         5      5
Į      Į                       1         1      1
Ķ      Ķ                       1         1      1
Ĺ      Ĺ                       2         2      2
Ļ      Ļ                       1         1      1
Ś      Ś                      54        54     54
...                          ...       ...    ...
훈      훈                       1         1      1
흡      흡                       1         1      1
흥      흥                       1         1      1
희      희                       7         7      7
히      히                       2         2      2
０      ０                       2         2      2
１      １                       3         3      3
２      ２                       3         3      3
３      ３                       1         1      1
４      ４                       1         1      1
５      ５                       1         1      1
９      ９                       3         3      3
Ｂ      Ｂ                       1         1      1
Ｃ      Ｃ                       1         1      1
Ｄ      Ｄ                       1         1      1
Ｊ      Ｊ                       1         1      1
Ｋ      Ｋ                       1         1      1
Ｏ      Ｏ                       1         1      1
Ｐ      Ｐ                       1         1      1
Ｑ      Ｑ                       1         1      1
Ｒ      Ｒ                       1         1      1
Ｓ      Ｓ                       1         1      1
ｈ      ｈ                       1         1      1
ｍ      ｍ                       2         2      2
ｎ      ｎ                       1         1      1
ｏ      ｏ                       5         5      5
ｒ      ｒ                       1         1      1
ｓ      ｓ                       1         1      1
ｔ      ｔ                       1         1      1
ｕ      ｕ                       1         1      1

[2989 rows x 3 columns]

In [27]:
single_char_verbatim_index = {}
for row in tmp.itertuples():
    arr = row.Index
    single_char_verbatim_index[arr[0]] = arr[1]

In [40]:
list(single_char_verbatim_index.items())[0:5]

[('#', 'number'), ('$', 'dollar'), ('%', 'percent'), ('&', 'and'), ('-', '-')]

In [42]:
len(all_data[(all_data['before'] == '#') & (all_data['after'] == 'number')])
len(all_data[(all_data['before'] == '#') & (all_data['after'] != 'number')])

len(all_data[(all_data['before'] == '&') & (all_data['after'] == 'and')])
len(all_data[(all_data['before'] == '&') & (all_data['after'] != 'and')])

1402

74

9684

0

In [49]:
len(all_data[(all_data['before'] == '#') & (all_data['after'] == 'number') & (all_data['class'] == 'VERBATIM')])
len(all_data[(all_data['before'] == '#') & (all_data['after'] != 'number') & (all_data['class'] == 'VERBATIM')])

1402

74

Verbatims are quite much like letters then?

### Save modified verbatim data

In [ ]:
chars_with_single_output_dict = {}
for c in chars_with_single_output:
    tmp = all_data_changed[all_data_changed['before'].str.contains(c, regex=False)]['after'].values[0]
    chars_with_single_output_dict[c] = tmp

In [254]:
all_data = all_data.reset_index(drop=True)

In [255]:
pickle.dump(all_data, open("data/en_train_verbatim_fixes_3.pkl", "wb"))

In [15]:
all_data = pickle.load(open("data/en_train_verbatim_fixes_3.pkl", "rb" ));

In [256]:
all_data_verb_stuff = all_data.copy()

# Data classes simplifying

In [261]:
all_data['class_org'] = all_data['class']

In [263]:
number_classes = ['DATE','CARDINAL','MEASURE','ORDINAL','DECIMAL','MONEY', 'DIGIT', 'TELEPHONE', 'TIME', 'FRACTION', 'ADDRESS']
all_data.loc[all_data['class'].isin(number_classes), 'class'] = 'NUMBERS'

In [267]:
all_data[all_data['class'] != 'PLAIN'].sample(5)

,sentence_id,token_id,class,before,after,class_org
9168258,692424,12,PUNCT,(,(,PUNCT
8878039,670967,8,PUNCT,",",",",PUNCT
2994446,231408,12,NUMBERS,3.8,three point eight,DECIMAL
851048,67769,5,PUNCT,),),PUNCT
4900191,374831,11,PUNCT,",",",",PUNCT


In [87]:
len(not_changed_cla_data)
len(not_changed_cla_data[all_data['before']!=not_changed_cla_data['after']])

9918139

699557

In [268]:
all_data.loc[all_data['before']==all_data['after'], 'class'] = 'NOT_CHANGED'

In [269]:
all_data.sample(5)

,sentence_id,token_id,class,before,after,class_org
1999580,155908,1,NOT_CHANGED,built,built,PLAIN
2114660,164688,8,NOT_CHANGED,final,final,PLAIN
2493500,193570,14,NOT_CHANGED,Crosswicks,Crosswicks,PLAIN
3193982,246472,7,NOT_CHANGED,Jesus,Jesus,PLAIN
454078,36434,21,NOT_CHANGED,(,(,PUNCT


In [271]:
all_data.groupby("class")["class"].count()

class
ELECTRONIC        4964
LETTERS         152986
NOT_CHANGED    9258648
NUMBERS         448172
PLAIN            36472
VERBATIM         16950
Name: class, dtype: int64

### Save

In [272]:
all_data = all_data.reset_index(drop=True)

In [273]:
pickle.dump(all_data, open("data/en_train_fixed_1.pkl", "wb"))

In [274]:
pickle.load(open("data/en_train_fixed_1.pkl", "rb" ));

# After words for not changed

In [9]:
all_data = pickle.load(open("data/en_train_fixed_1.pkl", "rb" ))

In [10]:
words_after_data = all_data[all_data['class'] != 'NOT_CHANGED']
len(all_data)
len(words_after_data)

9918192

659544

In [11]:
# arr = list(set(list(electronic_data['after'])))
# arr = list(set(list(sample_data['after'])))
arr = list(words_after_data['after'])
arr = [s.split(' ') for s in arr]
arr = np.concatenate(arr)
len(arr)
len(set(arr))

2066961

1346

In [21]:
words_after_count_index = {}
for w in arr:
    words_after_count_index[w] = words_after_count_index.get(w, 0) + 1

In [22]:
for i in [1, 5, 10, 50, 100, 250, 500, 1000, 10000]:
    tmp = [v for k, v in words_after_count_index.items() if v > i]
    print("{:>5d}: {:2.2%} (words: {})".format(i,  sum(tmp)/sum(words_after_count_index.values()), len(tmp)))

    1: 99.98% (words: 860)
    5: 99.93% (words: 554)
   10: 99.90% (words: 455)
   50: 99.69% (words: 270)
  100: 99.49% (words: 214)
  250: 99.05% (words: 155)
  500: 98.61% (words: 129)
 1000: 97.98% (words: 112)
10000: 84.02% (words: 54)


In [23]:
words_after_common = sorted(list(words_after_count_index.items()), key=lambda a: -a[1])
words_after_common = [k for k, v in words_after_common]
len(words_after_common)
words_after_common[0:5]

1346

['two', 'twenty', 'nineteen', 'thousand', 'one']

In [24]:
pickle.dump(words_after_common, open("data/en_features/words_after_all.pkl", "wb"))

In [41]:
i = 512
words_after_common[i]
words_after_count_index[words_after_common[i]]

'unfavorable'

7

In [42]:
i = 384
words_after_common[i]
words_after_count_index[words_after_common[i]]

'savior'

17

In [43]:
i = 256
words_after_common[i]
words_after_count_index[words_after_common[i]]

'tilde'

63

In [54]:
after_words_dict = dict((c, i) for i, c in enumerate(words_after_common[0:8192]))

In [55]:
after_words_dict['first']
after_words_dict['firsts']

51

280

# Testing some stuff

In [276]:
all_data.head()

,sentence_id,token_id,class,before,after,class_org
0,0,0,NOT_CHANGED,Brillantaisia,Brillantaisia,PLAIN
1,0,1,NOT_CHANGED,is,is,PLAIN
2,0,2,NOT_CHANGED,a,a,PLAIN
3,0,3,NOT_CHANGED,genus,genus,PLAIN
4,0,4,NOT_CHANGED,of,of,PLAIN


In [281]:
verb_changed = all_data[(all_data['class_org']=='VERBATIM') & (all_data['class']!='NOT_CHANGED')]

In [294]:
len(all_data[all_data['class']=='VERBATIM'])

16950

In [283]:
verb_changed = verb_changed.copy()

In [290]:
def manual_pre_checks(before):
    if chars_with_no_changes_re.search(before): #contains chars that were never changed
        #return before 
        return False
    if before in chars_with_single_output_dict:
        #return chars_with_single_output_dict[before]
        return False
    return True

In [288]:
manual_pre_checks("asdf")

False

In [325]:
verb_changed[verb_changed['before'].apply(manual_pre_checks)].sample(20)

,sentence_id,token_id,class,before,after,class_org
2460002,191016,5,VERBATIM,&,and,VERBATIM
831227,66233,8,VERBATIM,&,and,VERBATIM
4956471,379057,8,VERBATIM,&,and,VERBATIM
6433060,489303,2,VERBATIM,#,number,VERBATIM
5119171,391303,8,VERBATIM,&,and,VERBATIM
1235455,97391,9,VERBATIM,&,and,VERBATIM
7741826,586868,7,VERBATIM,#,number,VERBATIM
8453848,639758,13,VERBATIM,&,and,VERBATIM
3267504,252086,2,VERBATIM,&,and,VERBATIM
3735851,287485,6,VERBATIM,&,and,VERBATIM


In [327]:
all_data[(all_data['before'].str.contains('&')) & (all_data['after'] != 'and')]

,sentence_id,token_id,class,before,after,class_org
12116,992,17,LETTERS,Q&A,q and a,LETTERS
24296,1943,2,LETTERS,M&M's,m and m's,LETTERS
40534,3221,11,LETTERS,H&R,h and r,LETTERS
60702,4893,6,LETTERS,R&B,r and b,LETTERS
79553,6390,3,LETTERS,S&C,s and c,LETTERS
81530,6548,4,LETTERS,C&P,c and p,LETTERS
85802,6883,8,LETTERS,R&B,r and b,LETTERS
85894,6890,1,LETTERS,T&TEC,t and t e c,LETTERS
111104,8908,8,LETTERS,S&P,s and p,LETTERS
140611,11289,7,LETTERS,V&A,v and a,LETTERS


# Some debugging etc

In [ ]:
tmp = all_data[all_data['class']=='VERBATIM']
len(tmp)
len(tmp[tmp.before.str.contains(VERBATIM_CHAR)]) / len(tmp)

In [ ]:
tmp = all_data[all_data['class']=='LETTERS']
tmp[tmp['before'] == tmp['after']].sample(5)

In [ ]:
all_data[all_data['before'] == "::"].sample(5)

In [ ]:
all_data['class'].unique()

In [ ]:
all_data.groupby("class")["class"].count()

In [ ]:
all_data[all_data['class'] == "VERBATIM"].sample(100)